In [1]:
import tensorflow as tf
#print(tf.__version__)

2.8.0


In [2]:
import cv2
import numpy as np
import tensorflow as tf

In [7]:
def get_audio():
    model_dir = 'Saved Models\\AlexNet_Saved_Thu_Feb_10_22_02_19.json'
    model_weights_dir = 'Saved Models\\AlexNet_Saved_weights_Thu_Feb_10_22_02_19.hdf5'
    with open(model_dir, 'r') as json_file:
        json_saved_model = json_file.read()
    model = tf.keras.models.model_from_json(json_saved_model)
    model.load_weights(model_weights_dir) 
    model.compile(loss = 'categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    #model.summary()

    t,r,b,l=100,350,325,575 #size of window
    visual_dict={0:'1',1:'2',2:'3',3:'A',4:'B',5:'C',6:'J',7:'My',8:'Name',9:'Y'}

    import pyttsx3
    def text_audio(text):
        obj = pyttsx3.init()
        voices = obj.getProperty('voices')
        obj.setProperty("voice",voices[1].id)
        obj. setProperty("rate", 150)
        speech = text
        obj.say(speech)
        obj.runAndWait()
    camera = cv2.VideoCapture(0) #Created object 
    num_frames = 0

    while(True):
        # Capture the video frame by frame
        success, frame = camera.read()
        frame = cv2.flip(frame, 1)
        roi = frame[140:380, 235:440]
        cv2.rectangle(frame, (225,135), (450,400), (0,255,0), 2)
        clone=frame.copy()


        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        lh = 0
        ls = 0
        lv = 0
        uh = 255
        us = 32
        uv = 255

        cv2.putText(clone, "Put your hand gestures in the rectangle", (5, 25), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        cv2.putText(clone, "Press 'q' to exit.", (5, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
        #Storing the lower and upper bound in an array
        l_b = np.array([lh, ls, lv])
        u_b = np.array([uh, us, uv])

        mask = cv2.inRange(hsv, l_b, u_b)
        mask = cv2.bitwise_not(mask)
        res = cv2.bitwise_and(roi, roi, mask=mask)
        res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)

        (cnts, _) = cv2.findContours(res, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts)>0:
            # Storing the max contors (Which area is maximum)
            max_cont = max(cnts, key=cv2.contourArea)
            if max_cont is not None:
                mask = cv2.drawContours(res, [max_cont + (r, t)], -1, (0, 0, 255))
                mask = np.zeros(res.shape, dtype="uint8")
                cv2.drawContours(mask, [max_cont], -1, 255, -1)
                #cv2.imshow('1_normal mask', mask)
                res = cv2.bitwise_and(res, res, mask=mask)
                #cv2.imshow('2_res', res)
                high_thresh, thresh_im = cv2.threshold(res, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                lowThresh = 0.5 * high_thresh
                res = cv2.Canny(res, lowThresh, high_thresh)
                cv2.imshow('Canny', res)
                # Prediction
                if res is not None and cv2.contourArea(max_cont) > 1000:
                    final_res = cv2.resize(res, (224,224))
                    final_res = final_res / 255
                    final_res = final_res.reshape(-1, 224, 224, 1)
                    output = model.predict(final_res)
                    prob = np.amax(output)
                    sign = np.argmax(output)#gives the index of maximum vale of in array
                    final_sign = visual_dict[sign]
                    text_audio(final_sign)
                    cv2.putText(clone, 'Sign : ' + str(final_sign), (175, 90), cv2.FONT_HERSHEY_COMPLEX, 2,(0, 0, 255))
        num_frames += 1
        cv2.imshow("Webcam", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF
        # if the user pressed "q", then stop looping
        if keypress == ord("q") or keypress == ord("Q"):
            break

    camera.release()
    cv2.destroyAllWindows()

In [8]:
#get_audio()